In [88]:
from scapy.all import PcapReader
import re
from typing import List, Any
from scapy.plist import PacketList
import glob
from tqdm import tqdm   
import pandas as pd
import csv

In [89]:
# read the pcap file and return a list of packets
def read_pcap(filename: str) -> PacketList:
	reader = PcapReader(filename)
	return reader.read_all()

In [20]:
# get the number of packets of each packet size occuring in the dataset
PATH = "data/*.pcap"
files = sorted(glob.glob(PATH)) 
packet_sizes_dict = {}
for file in tqdm(files):
    plist = read_pcap(file)
    for packet in plist:
        if len(packet) not in packet_sizes_dict:
            packet_sizes_dict[len(packet)] = 1
        else:
            packet_sizes_dict[len(packet)] = packet_sizes_dict[len(packet)] + 1

100%|██████████| 1500/1500 [02:12<00:00, 11.29it/s]


In [104]:
# get the top K packet sizes to later use them as features
K = 20
sorted_dict = sorted(packet_sizes_dict.items(), key = lambda item: item[1], reverse=True)
TOP_K_INDICES, _ = zip(*sorted_dict[:K])

In [105]:
""" Feature extraction helper functions """
# get label name
def extract_label_from_filename(filename: str) -> int:
	return int(re.search('grid_(\d+).+', filename).group(1))

# get number of packets
def number_of_packets(plist: PacketList) -> int:
    return len(plist)

# get time duration
def time_duration(plist: PacketList) -> float:
    max_timestamp = max([packet.time for packet in plist])
    min_timestamp = min([packet.time for packet in plist])
    return float(max_timestamp - min_timestamp)

# get total traffic volume
def total_traffic_volume(plist: PacketList) -> int:
    return sum([len(packet) for packet in plist])

# get number of packets sent by client
def count_packets_sent_by_client(plist: PacketList) -> PacketList:
    return len([packet for packet in plist if packet[0][1].src.startswith('172.')])

# get number of packets sent to the client
def count_packets_sent_to_client(plist: PacketList) -> PacketList:
    return len([packet for packet in plist if packet[0][1].dst.startswith('172.')])

# count the number of packets with length in top k
def count_packets_with_length_in_top_k(plist: PacketList, top_k_indices) -> int:
    packet_count = {index: 0 for index in top_k_indices}
    for packet in plist:
        if len(packet) in packet_count:
            packet_count[len(packet)] = packet_count[len(packet)] + 1
    return packet_count

In [92]:
# extract features
def extract_features(filename: str) -> List[Any]:
    # read pcap file
    plist = read_pcap(filename)
    # extract features
    features = {}
    features['label'] = extract_label_from_filename(filename)
    features['number_of_packets'] = number_of_packets(plist)
    features['time_duration'] = time_duration(plist)
    features['total_traffic_volume'] = total_traffic_volume(plist)
    features['count_packets_sent_by_client'] = count_packets_sent_by_client(plist)
    features['count_packets_sent_to_client'] = count_packets_sent_to_client(plist)
    for index, count in count_packets_with_length_in_top_k(plist, TOP_K_INDICES).items():
        features[f'count_packets_with_length_{index}'] = count
    return features

In [93]:
extract_features("data/grid_1_20230527_095444.pcap")

{'label': 1,
 'number_of_packets': 437,
 'time_duration': 18.438893,
 'total_traffic_volume': 461872,
 'count_packets_sent_by_client': 181,
 'count_packets_sent_to_client': 256,
 'count_packets_with_length_54': 183,
 'count_packets_with_length_590': 79,
 'count_packets_with_length_2530': 60,
 'count_packets_with_length_2950': 0,
 'count_packets_with_length_1292': 26,
 'count_packets_with_length_3674': 5,
 'count_packets_with_length_3768': 17,
 'count_packets_with_length_4398': 0,
 'count_packets_with_length_1198': 20,
 'count_packets_with_length_1502': 0,
 'count_packets_with_length_1104': 3,
 'count_packets_with_length_778': 0,
 'count_packets_with_length_5006': 5,
 'count_packets_with_length_496': 4,
 'count_packets_with_length_4124': 1,
 'count_packets_with_length_7294': 0,
 'count_packets_with_length_1314': 0,
 'count_packets_with_length_380': 2,
 'count_packets_with_length_5846': 0,
 'count_packets_with_length_402': 1,
 'count_packets_with_length_6244': 1,
 'count_packets_with_len

In [106]:
# extract features
feature_list = []
for filename in tqdm(files):
    feature_list.append(extract_features(filename))

100%|██████████| 1500/1500 [04:02<00:00,  6.19it/s]


In [107]:
# write features to csv file
keys = feature_list[0].keys()
with open('features.csv', 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(feature_list)